# Dash_Services

In [ ]:
%reload_ext autoreload
%autoreload 2
import colabexts
from colabexts.jcommon import *

jpath=os.path.dirname(colabexts.__file__)
jcom = f'{jpath}/jcommon.ipynb'
%run $jcom

import os, sys, datetime, re, json, importlib
from collections import defaultdict
from sys import modules
from IPython.display import HTML, Javascript
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

%matplotlib inline
pd.set_option('display.max_colwidth', 40)
pd.set_option('display.max_rows', 16)

import logging
for handler in logging.root.handlers[:]: logging.root.removeHandler(handler)
logging.basicConfig( level=logging.INFO,
        format='%(asctime)s %(name)s %(levelname)s: %(message)s',
        #handlers=[ logging.FileHandler("/tmp/stream.log"), logging.StreamHandler()],
        handlers=[ logging.StreamHandler()],
)
logger = logging.getLogger( "geoapp" )
logger.info( f"Testing logging" )

2023-07-14 17:15:27,848 geoapp INFO: Tesrting logging


# dash_services

In [1]:
%%writefile ../dash_services.py
#!/usr/bin/env python 

#*** DO NOT EDIT - GENERATED FROM tseries/notebooks/tseries_services.ipynb ****

import sys, os,  glob, logging
from  mangorest.mango import webapi

sys.path.append("../..")

MBASE  = "/opt/data/data/dashboards/"
logger = logging.getLogger( "geoapp" )

#---------------------------------------------------------------------------------    
@webapi("/dashboard/save_dashboard")
def save_dashboard( name, contents, **kwargs):
    name = name or "default"
    if (name.endswith(".json")):
        name = name[:-5]
        
    dst = f"{MBASE}/{name}.json"
    dstd= os.path.dirname(dst)
    if not os.path.exists(dstd ):
        os.makedirs(dstd)
    
    with open(dst, "w") as f:
        f.write(contents.strip())

    return f"Saved {dst}"    
#---------------------------------------------------------------------------------    
@webapi("/dashboard/get_dashboard")
def get_dashboard( name, **kwargs):
    if (name.endswith(".json")):
        name = name[:-5]
    dst = f"{MBASE}/{name}.json"
    if not os.path.exists(os.path.dirname(dst) ):
        os.makedirs(dst)
    
    ret = open(dst, "r").read()
    return ret  
#------------------------------------------------------------------------------
@webapi("/dashboard/delete_dashboard")
def delete_dashboard(name="", **kwargs):
    dst = f"{MBASE}/{name}.json"
    if ( not os.path.exists(dst) ):
        return "OK {dst} does not exist!!"
    os.remove(dst)

    return f"Deleted {dst}."

#------------------------------------------------------------------------------
@webapi("/dashboard/getall_dashboards")
def getAllDashboards( nrows=10000, patt='*.json', **kwargs):
    
    files = glob.glob(f"{MBASE}{patt}")
    files.sort(key=os.path.getmtime, reverse=True)

    ret = {
        "name": "dashboards",
        "columns": ["name"],
        "values": [[f[len(MBASE):]] for f in files][0:nrows]
    }
    return ret


Overwriting ../dash_services.py


In [22]:
files = glob.glob(f"{MBASE}*.json")
files.sort(key=os.path.getmtime, reverse=True)
files

['/opt/data/data/dashboards/default.json',
 '/opt/data/data/dashboards/sada.json']

In [20]:
type(files[0])

str